# Regularization

In [2]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Data

In [5]:
from common.data import get_data

X, Y, demographics = get_data()
ages, sexes = demographics['Age'], demographics['Sex']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Sex: {sexes.shape}')

X: (65, 34716) | Y: 16 | Age: (65,) | Sex: (65,)


In [10]:
from scipy import stats
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import RepeatedKFold, cross_validate, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Run for one target, one age bin

In [70]:
selected_target = "WISC_PSI"
y = Y[selected_target]

print(f'y: {y.shape}')

y: (51,)


In [71]:
from common.binning import bin_by_age

bins = bin_by_age(X, y, ages, y)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

Bin 0 Range: 6.22 -> 8.80
Bin 1 Range: 9.34 -> 11.97
Bin 2 Range: 12.50 -> 15.87
---
Bin 0 Range: 80.00 -> 144.00
Bin 1 Range: 86.00 -> 123.00
Bin 2 Range: 66.00 -> 126.00
---
Bin 1: (19, 34716) | Bin 2: (16, 34716) | Bin 3: (16, 34716)


### Set up the model pipeline, metrics, and cross-validation approach

In [54]:
def regression_scorer(reg, X, y):
    y_pred = reg.predict(X)
    return stats.pearsonr(y, y_pred)[0]

scoring = ['train_score', 'test_score']
rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=251183)

In [78]:
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y

estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], 
                                        scoring=regression_scorer, cv=rkf)]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cv = pipe['ridgecv']

print(f'Best alpha: {ridge_cv.alpha_}')
print(f'Best score: {ridge_cv.best_score_:.2f}')

estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

Best alpha: 50000
Best score: 0.10


### Run cross-validation

In [8]:
%%time
X_cv = X
y_cv = y

scores = cross_validate(pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_jobs=-1, 
                        return_train_score=True, return_estimator=True)

coefs = [estimator['ridge'].coef_ for estimator in scores['estimator']]
avg_coef = np.mean(coefs, axis=0)

print(f'Target: {selected_target}')
for metric in scoring:
    metric_values = scores[metric]
    print(f'Avg {metric}: {np.mean(metric_values):.3f}')

Target: WISC_FSIQ
Avg train_r2: 0.641
Avg test_r2: 0.144
Avg test_pearsonr: 0.405
Avg test_spearmanr: 0.419
CPU times: user 1.82 s, sys: 1.14 s, total: 2.96 s
Wall time: 26.3 s


In [9]:
from common.paths import RIDGE_WEIGHTS

np.save(RIDGE_WEIGHTS, avg_coef)

### Run permutation statistic

In [79]:
%%time
score, _, pvalue = permutation_test_score(
    pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=3000, n_jobs=-1)

print(f'Target: {selected_target}')
print(f'Alpha: {ridge_cv.alpha_} | Score: {score:.2f} | p-value: {pvalue:.4f}')

Target: WISC_PSI
Alpha: 50000 | Score: 0.10 | p-value: 0.3382
CPU times: user 31.6 s, sys: 1.31 s, total: 32.9 s
Wall time: 3min 3s


## Run for all targets, all age bins

In [8]:
def regression_scorer(reg, X, y):
    y_pred = reg.predict(X)
    return stats.pearsonr(y, y_pred)[0]

scoring = ['train_score', 'test_score']
rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=251183)

In [ ]:
%%time
from common.binning import bin_by_age
from common.wisc import FSIQ, PRIMARY_INDICES

targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    bins = bin_by_age(X, y, ages, y)
    bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
    X_all = [X, bin_1[0], bin_2[0], bin_3[0]]
    y_all = [y, bin_1[1], bin_2[1], bin_3[1]]
    age_bin_label = ["All  ", "Bin 1", "Bin 2", "Bin 3"]
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, age_bin_label):
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 70000, 5000)], 
                                        scoring=regression_scorer, cv=rkf)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        
        # Do permutation test
        estimators = [StandardScaler(), Ridge(alpha=pipe['ridgecv'].alpha_)]
        pipe = make_pipeline(*estimators)
        
        score, _, pvalue = permutation_test_score(
            pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=3000, n_jobs=-1)
        print(f'Bin: {bin_label} | Target: {target} | Alpha: {ridge_cv.alpha_} | Score: {score:.2f} | p-value: {pvalue:.4f}')
    print('---')